# This notebook is used to create the main interfaces for CMP3-EEG part

## This is the backbone used to create the interfaces

In [1]:
from nipype.interfaces.base import BaseInterfaceInputSpec, BaseInterface, File, TraitedSpec, traits

# Define function
def my_function(input1,input2,out_file):
    import numpy as np
    
    print('Did nothing but print this')
    

class MyFunctionInputSpec(BaseInterfaceInputSpec):
    in_file = File(exists=True, mandatory=True, desc='the input image')
    out_file = File(mandatory=True, desc='the output image') # Do not set exists=True !!
    a_random_list_parameter = traits.List([50.0, 0.0, 0.0], traits.Float, usedefault=True,
                              desc='the translation component of the input transform')

class MyFunctionOutputSpec(TraitedSpec):
    out_file = File(desc='the output image')

class MyFunction(BaseInterface):
    input_spec = MyFunctionInputSpec
    output_spec = MyFunctionOutputSpec

    def _run_interface(self, runtime):

        # Call our python code here:
        my_function(
            self.inputs.in_file,
            self.inputs.a_random_list_parameter,
            self.inputs.out_file
        )

        # And we are done
        return runtime
    
    def _list_outputs(self):
        return {'out_file': self.inputs.out_file}

In [2]:
this_works = MyFunction(
    in_file='README.md',
    out_file='OUT_file.npy')
result = this_works.run()


Did nothing but print this


## The main nodes are:
#### 1) EEG-Preprocessing
#### 2) Lead Field matrix
#### 3) Inverse solution
#### 4) Parcellation
#### 5) dFC
#### 6) post-processing

## 1) Create node for eeg-preprocessing

## 2) Create node for LeadField matrix




#### 2.1. Create bem surfaces and visualize them


In [2]:
from nipype.interfaces.mne import WatershedBEM
from nipype import Node
import os



#### 2.2. Coregistration --> Create trans.fif fil


In [4]:
from nipype.interfaces.base import CommandLine, CommandLineInputSpec, Directory

class CoregTransInputSpec(CommandLineInputSpec):
    subject_dir = Directory(exists=True, mandatory=True, argstr='%s',
                   position=0, desc='the input description')

class CoregTrans(CommandLine):
    _cmd = 'mne coreg'
    input_spec = CoregTransInputSpec
    def _run_interface(self, runtime):
        runtime = super(CoregTrans, self)._run_interface(runtime)
        return runtime

CoregTrans.help()



Wraps the executable command ``mne coreg``.

Inputs::

        [Mandatory]
        subject_dir: (a pathlike object or string representing an existing
                  directory)
                the input description
                argument: ``%s``, position: 0

        [Optional]
        args: (a unicode string)
                Additional parameters to the command
                argument: ``%s``
        environ: (a dictionary with keys which are a bytes or None or a value
                  of class 'str' and with values which are a bytes or None or a
                  value of class 'str', nipype default value: {})
                Environment variables

Outputs::

        None



#### 2.3. Visualization of the corregistration


In [5]:
from nipype.interfaces.base import BaseInterfaceInputSpec, BaseInterface, File, TraitedSpec, traits, Str
import mne

def vis_coreg(sub_id,soft_id):
    
    if soft_id.lower() == "cartool":
        datafold = "/home/jo8558/data/SINERGIA_2019/"##### TO DO #####
        montage_fname = os.path.join(datafold,'02_ESI_CARTOOL','CartoolFiles',sub_id,sub_id+'.xyz')
        dev_head_t_fname = os.path.join(datafold,'02_ESI_CARTOOL','CartoolFiles',sub_id,'More',sub_id+'.Transform.Electrodes Coregistration.Electrodes to Original MRI.txt')
        dev_head_t = np.loadtxt(dev_head_t_fname)

        n = int(open(montage_fname).readline().lstrip().split(' ')[0])
        all_coord = np.loadtxt(montage_fname, skiprows=1, usecols=(0, 1, 2), max_rows=n)
        all_names = np.loadtxt(montage_fname, skiprows=1, usecols=3, max_rows=n,dtype=np.dtype(str)).tolist()
        all_coord = list(map(lambda x: x/1000,all_coord))
        
        nas_coord = all_coord[all_names.index('nasion')]
        lpa_coord = all_coord[all_names.index('lpa')]
        rpa_coord = all_coord[all_names.index('rpa')]
        ch_coord  = [all_coord[idx] for idx, chan in  enumerate(all_names) if chan not in ['lpa','rpa','nasion']]
        ch_names  = [all_names[idx] for idx, chan in  enumerate(all_names) if chan not in ['lpa','rpa','nasion']]

        montage = mne.channels.make_dig_montage(ch_pos=dict(zip(ch_names, ch_coord)),nasion=nas_coord,rpa=rpa_coord,lpa=lpa_coord,coord_frame='head')

        montage.dev_head_t = dev_head_t
        fif_fname = os.path.join(datafold,'MNE_tmp',sub_id,sub_id+'-montage.fif')
        montage.save(fif_fname)        
        info = mne.create_info(ch_names, 250., 'eeg', montage=montage)
    else:
        raise NotImplementedError()
class VisCoregInputSpec(BaseInterfaceInputSpec):
    subject_id = Str(mandatory=True, desc='the subject id')
    software_id = Str(default_value="cartool", desc='the software used to save electrode coordinates name')

class VisCoregOutputSpec(TraitedSpec):
    info = traits.Instance(mne.Info)
    
class VisCoreg(BaseInterface):
    input_spec = VisCoregInputSpec
    output_spec = VisCoregOutputSpec
    def _run_interface(self, runtime):

        vis_coreg(
            self.inputs.subject_id,
            self.inputs.software_id
        )
        return runtime

    
VisCoreg.help()

Inputs::

        [Mandatory]
        subject_id: (a unicode string)
                the subject id

        [Optional]
        software_id: (a unicode string)
                the software used to save electrode coordinates name

Outputs::

        info: (an Info or None)



#### 2.4. Compute source space


In [6]:
from nipype.interfaces.base import BaseInterfaceInputSpec, BaseInterface, File, TraitedSpec, traits, Str
import mne
# Define function
def source_space(sub_id,subjects_dir, soft_id,info,mindist=5.0,n_jobs=4):
    if soft_id.lower() != "cartool": # then use MNE
        datafold = "/home/jo8558/data/SINERGIA_2019/"##### TO DO #####
        fwd_fname = os.path.join(datafold,'MNE_tmp','sub-'+subject,'fwd.fif')
        trans_fname = os.path.join(datafold,'MNE_tmp','sub-'+subject,'sub-'+subject+'-trans.fif')
            
        if not(os.path.exists(fwd_fname)):
            conductivity = (0.3, 0.006, 0.3)  # for three layers
            for elem in ["inner_skull","outer_skull","outer_skin"]:
                if (elem not in os.listdir(os.path.join(subjects_dir,sub_id,'bem'))) and ("watershed" in os.listdir(os.path.join(subjects_dir,sub_id,'bem'))):
                    cmd = 'cp '+ os.path.join(subjects_dir,subject,'bem','watershed','*'+elem+'*') + ' ' + os.path.join(subjects_dir,sub_id,'bem',elem+'.surf')
                    os.system(cmd)    

            model = mne.make_bem_model(subject=sub_id, ico=4,
                                       conductivity=conductivity,
                                       subjects_dir=subjects_dir)

            bem = mne.make_bem_solution(model)

            fwd = mne.make_forward_solution(info, trans=trans_fname, src=src,
                                            bem=bem, eeg=True, mindist=mindist, n_jobs=n_jobs)
            mne.write_forward_solution(fwd_fname, fwd, overwrite=True, verbose=None)
        else:
            fwd = mne.read_forward_solution(fwd_fname)
    
class SourceSpaceInputSpec(BaseInterfaceInputSpec):
    subject_id = Str(mandatory=True, desc='the subject id')
    subjects_dir = Directory(exists=True, mandatory=True, argstr='%s',
                   position=0, desc='subject directory')
    software_id = Str(default_value="cartool", desc='the software used to save electrode coordinates name')
    # TODO: Decide whether or not make these mandatory: 
    info = traits.Instance(mne.Info)
    #bem = traits.Instance(mne.bem.ConductorModel)
    mindist = traits.Float
    n_jobs = traits.Int
    
class SourceSpace(BaseInterface):
    input_spec = SourceSpaceInputSpec

    def _run_interface(self, runtime):

        # Call our python code here:
        source_space(
            self.inputs.subject_id,
            self.inputs.software_id,
            self.inputs.info,
            self.inputs.bem,
            self.inputs.mindist,
            self.inputs.n_jobs
        )

        # And we are done
        return runtime
    
SourceSpace.help()

Inputs::

        [Mandatory]
        subject_id: (a unicode string)
                the subject id
        subjects_dir: (a pathlike object or string representing an existing
                  directory)
                subject directory
                argument: ``%s``, position: 0

        [Optional]
        software_id: (a unicode string)
                the software used to save electrode coordinates name
        info: (an Info or None)
        mindist: (a float)
        n_jobs: (an integer (int or long))

Outputs::

        None



#### 2.5. Compute forward solution


In [ ]:
EEG_path = os.path.join(datafold,'01_EEG_PREP','Data_FaceTask',subject,subject+'_FACES_250HZ_prepd.set') 

epochs = mne.read_epochs_eeglab(EEG_path, events=None, event_id=None, eog=(), verbose=None, uint16_codec=None)
epochs.set_montage(montage,raise_if_subset=False)
epochs.apply_baseline((0,None))
epochs.set_eeg_reference('average',projection = True)
epochs.apply_proj()

cov_fname = os.path.join(datafold,"MNE_tmp",'sub-'+subject,"noise_cov.fif")
if os.path.exists(cov_fname):
    noise_cov = mne.read_cov(cov_fname)
else:
    noise_cov = mne.compute_covariance(
    epochs, tmax=0., method=['shrunk', 'empirical'], rank=None, verbose=True)
    mne.write_cov(cov_fname,noise_cov)
    
inverse_operator = mne.minimum_norm.make_inverse_operator(epochs.info, fwd, noise_cov)

method = "dSPM"
#method = "eLORETA"
snr = 3.
lambda2 = 1. / snr ** 2

print('Computing SOURCE data faces')
stc_faces,residual_faces = mne.minimum_norm.apply_inverse(evoked_faces, inverse_operator, lambda2,
                              method=method, pick_ori=None,return_residual=True,
                              verbose=True)
print('Computing SOURCE data scra')
stc_scra,residual_scra = mne.minimum_norm.apply_inverse(evoked_scra, inverse_operator, lambda2,
                              method=method, pick_ori=None,return_residual=True,
                              verbose=True)


#### Create all nodes

In [7]:

sub_id = 'sub-01'
subjects_dir = os.path.join("/home/jo8558/data/SINERGIA_2019",'03_ANATOMY','freesurfer','subjects')

# Create bem surfaces
bem_surfaces = Node(WatershedBEM(subject_id = sub_id,
                                 subjects_dir = subjects_dir),
               name="bem_surfaces")

# Coregistration - Create transormation file
coregistration = Node(CoregTrans(subject_dir = os.path.join(subjects_dir,sub_id)),
               name="coregistration")

# Visualization the coregistration 
vis_coreg = Node(VisCoreg(subject_id = sub_id, 
                         software_id = "Cartool"),
               name="vis_coreg")

# Compute Source Space
sourcespace = Node(SourceSpace(subject_id= sub_id,
                               subjects_dir = subjects_dir,
                               software_id= "mne",
                               info = vis_coreg.outputs.info
                              ),
                  name="sourcespace")


## 3) Create node for Inverse solution

## 4) Create node for Parcellation

## 5) Create node for Post-processing